In [ ]:
from pathlib import Path
import sqlite3

In [ ]:

SQLITE_DB_PATH = Path("~/Library/Group Containers/group.net.whatsapp.WhatsApp.shared/ChatStorage.sqlite").expanduser()
db = sqlite3.connect(str(SQLITE_DB_PATH))


In [24]:
start_of_week_str

'2025-07-14 00:00:00'

In [55]:
from datetime import datetime, timezone, timedelta

# Function: Seconds since Jan 1, 2001 UTC (WhatsApp/iOS epoch)
def get_timestamp_since_2001(dt):
    epoch_2001 = datetime(2001, 1, 1, tzinfo=timezone.utc)
    return int((dt - epoch_2001).total_seconds())

# Function: Convert WhatsApp timestamp to datetime
def whatsapp_timestamp_to_datetime(timestamp):
    epoch_2001 = datetime(2001, 1, 1, tzinfo=timezone.utc)
    return epoch_2001 + timedelta(seconds=timestamp)

now = datetime.now(timezone.utc)
start_of_week_date = (now - timedelta(days=now.weekday())).date()
start_of_week = datetime.combine(start_of_week_date, datetime.min.time(), tzinfo=timezone.utc)
start_of_week_timestamp = get_timestamp_since_2001(start_of_week)


In [56]:
start_of_week_timestamp

774144000

In [ ]:
import datetime

# Get the start of the current week (Monday)



query = """
SELECT
    ZWACHATSESSION.ZCONTACTJID as chat_jid,
    ZWAMESSAGE.ZTEXT as message_text,
    ZWAMESSAGE.ZMESSAGEDATE as message_date
FROM
    ZWACHATSESSION
JOIN
    ZWAMESSAGE ON ZWAMESSAGE.ZCHATSESSION = ZWACHATSESSION.Z_PK
WHERE
    ZWAMESSAGE.ZMESSAGEDATE >= ? -- WhatsApp stores dates as unix timestamps
ORDER BY
    chat_jid, message_date
"""

# WhatsApp's ZMESSAGEDATE is often in Mac absolute time (seconds since 2001-01-01)
# If so, you may need to adjust the query and conversion accordingly.
# For now, let's assume it's unix time (seconds since 1970-01-01).
# If not, you may need to add (strftime('%s', ?) - strftime('%s', '2001-01-01 00:00:00')) to convert.

cursor = db.cursor()
cursor.execute(query, (start_of_week_timestamp,))
results = cursor.fetchall()

In [59]:
len(results)

301

In [63]:

from collections import defaultdict

channels = defaultdict(list)
for channel_name, message_text, message_date in results:
    channels[channel_name].append({
        "message_text": message_text,
        "message_date": message_date,
    })

for channel, messages in channels.items():
    if not messages:
        continue
    
    print(f"Channel: {channel}")
    for msg in messages:
        print(f"  [{msg['message_date']}] {msg['message_text']}")
    print("-" * 40)


Channel: 120363023426092133@g.us
  [774282627.328522] None
----------------------------------------
Channel: 120363023686310544@g.us
  [774217309] Huh… What’s that smell? You feel it, Ieva?
  [774217322] Hmmm what is it Alan?
  [774217361] I don’t know, it smells like, AN EFFING COOL ROOFTOP SALSA PARTY AGAIN?!?!?!
  [774217381] None
  [774217448] Guess my senses were right!
  [774223691] *LAST MINUTE CALL*  💃🏽🕺

Who can join us for some dancing in The Hague today?! 

*Looking for 2 followers and 1 leader.* 

You can drive with us from A’dam Central Station or HQ, at 16:15! 

🎈 *What’s the plan?* 
We’ll bring vibes, fun, and entertainment to a graduation evening at a school! Salsa dances, flashmob ruedas, maybe a merengue parade with the crowd… 

DJ Manga & Yunan will be there spinning the tunes with live percussion! 🪘🎶 

You don't have to be the best dancer or performer, if you enjoy creating a fun atmosphere and making people smile, you're EXACTLY who we need!

📆*Date:* 15 July
🕟*Tim

In [13]:
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table_name_tuple in tables:
    table_name = table_name_tuple[0]
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info('{table_name}');")
    columns = cursor.fetchall()
    print("Columns:")
    for col in columns:
        print(f"  {col[1]} ({col[2]})")
    print("-" * 40)


Table: ZWABLACKLISTITEM
Columns:
  Z_PK (INTEGER)
  Z_ENT (INTEGER)
  Z_OPT (INTEGER)
  ZJID (VARCHAR)
----------------------------------------
Table: ZWACHATPROPERTIES
Columns:
  Z_PK (INTEGER)
  Z_ENT (INTEGER)
  Z_OPT (INTEGER)
  ZALERTS (INTEGER)
  ZENABLED (INTEGER)
  ZCHATSESSION (INTEGER)
  ZMUTEDATE (TIMESTAMP)
  ZSOUNDNAME (VARCHAR)
----------------------------------------
Table: ZWACHATPUSHCONFIG
Columns:
  Z_PK (INTEGER)
  Z_ENT (INTEGER)
  Z_OPT (INTEGER)
  ZALERTS (INTEGER)
  ZMUTEDUNTIL (TIMESTAMP)
  ZJID (VARCHAR)
  ZRINGTONE (VARCHAR)
  ZSOUND (VARCHAR)
----------------------------------------
Table: ZWACHATSESSION
Columns:
  Z_PK (INTEGER)
  Z_ENT (INTEGER)
  Z_OPT (INTEGER)
  ZARCHIVED (INTEGER)
  ZCONTACTABID (INTEGER)
  ZFLAGS (INTEGER)
  ZHIDDEN (INTEGER)
  ZIDENTITYVERIFICATIONEPOCH (INTEGER)
  ZIDENTITYVERIFICATIONSTATE (INTEGER)
  ZMESSAGECOUNTER (INTEGER)
  ZREMOVED (INTEGER)
  ZSESSIONTYPE (INTEGER)
  ZSPOTLIGHTSTATUS (INTEGER)
  ZUNREADCOUNT (INTEGER)
  ZGROU